In [1]:
import pandas as pd
import pickle
import numpy as np
import re

In [2]:
df = pd.read_csv("../DataSet/douban_boutique/movie.csv", encoding='gbk')
df.head()

,title,types,score,director,writer,actor,account,country,label,flim_version
0,这个杀手不太冷,"剧情, 动作, 犯罪",9.4,吕克·贝松 Luc Besson,吕克·贝松 Luc Besson,"让·雷诺, 娜塔莉·波特曼, 加里·奥德曼, 丹尼·爱罗, 彼得·阿佩尔, 迈克尔·巴达鲁科...",1088567,法国,1023.25298,2
1,七武士,"动作, 冒险, 剧情",9.2,黑泽明 Akira Kurosawa,"黑泽明 Akira Kurosawa, 小国英雄 Hideo Oguni, 桥本忍 Shin...","三船敏郎, 志村乔, 稻叶义男, 宫口精二, 千秋实, 加东大介, 木村功",91163,日本,83.86996,6
2,蝙蝠侠：黑暗骑士,"剧情, 动作, 科幻, 犯罪, 惊悚",9.1,克里斯托弗·诺兰 Christopher Nolan,"乔纳森·诺兰 Jonathan Nolan, 克里斯托弗·诺兰 Christopher No...","克里斯蒂安·贝尔, 希斯·莱杰, 艾伦·艾克哈特, 迈克尔·凯恩, 玛吉·吉伦哈尔, 加里·...",423487,"美国, 英国",385.37317,1
3,指环王3：王者无敌,"剧情, 动作, 奇幻, 冒险",9.1,彼得·杰克逊 Peter Jackson,"弗兰·威尔士 Fran Walsh, Philippa Boyens, 彼得·杰克逊 Pet...","维果·莫腾森, 伊莱贾·伍德, 西恩·奥斯汀, 丽芙·泰勒, 伊恩·麦克莱恩, 奥兰多·布鲁...",350209,"美国, 新西兰",318.69019,3
4,搏击俱乐部,"剧情, 动作, 悬疑, 惊悚",9.0,大卫·芬奇 David Fincher,"恰克·帕拉尼克 Chuck Palahniuk, Jim Uhls","爱德华·诺顿, 布拉德·皮特, 海伦娜·伯翰·卡特, 扎克·格雷尼尔, 米特·洛夫, 杰瑞德...",465674,"美国, 德国",419.10660,1


In [31]:
type_enum = ["黑色电影","恐怖","悬疑","同性","犯罪","儿童","Reality-TV","喜剧","惊栗","歌舞","科幻","家庭",
             "爱情","Adult","情色","灾难","音乐","运动","鬼怪","历史","冒险","西部","惊悚","奇幻","传记",
             "武侠","古装","战争","动作","剧情","悬念","动画"]

type_enum = sorted(type_enum, key = lambda kv:len(kv))
type_enum

['恐怖',
 '悬疑',
 '同性',
 '犯罪',
 '儿童',
 '喜剧',
 '惊栗',
 '歌舞',
 '科幻',
 '家庭',
 '爱情',
 '情色',
 '灾难',
 '音乐',
 '运动',
 '鬼怪',
 '历史',
 '冒险',
 '西部',
 '惊悚',
 '奇幻',
 '传记',
 '武侠',
 '古装',
 '战争',
 '动作',
 '剧情',
 '悬念',
 '动画',
 '黑色电影',
 'Adult',
 'Reality-TV']

In [32]:
with open('../DataSet/douban_boutique/type_list.pkl','wb') as f:
    pickle.dump(type_enum, f)

### type-director

In [46]:
type_director_dict = dict()

for t in type_enum: # 对每种类型
    dir_dict = dict() # 清空[导演-电影数(小于7 || 7-8 || 大于8)字典]
    for i in range(df.shape[0]): # 对每一条条目
        typestr = df.iloc[i,1] # 类型字符串
        if re.search(t, typestr) is not None: # 如果不是这个类型
            continue # 就跳过
        dirstr = df.iloc[i,3] # 这个电影的导演
        if type(dirstr) is not str: # 这里确保字段是字符串
            continue
        score = df.iloc[i,2] # 这个电影的得分
        sidx = 0 if score<7 else (1 if score<=8 else 2) # 根据得分知道要在哪个下标位置+1
        # 写入[导演-电影数字典]
        if dirstr in dir_dict:
            dir_dict[dirstr][sidx] += 1
        else:
            dir_dict[dirstr] = [0,0,0]
            dir_dict[dirstr][sidx] += 1
    # 对这个[导演-电影数]字典按电影总数从大到小排序，并只取top4
    sort_list = sorted(dir_dict.items(), key = lambda kv:(sum(kv[1])), reverse=True)[:4]
    # 将四个形如('王晶 Jing Wong', [26, 11, 0])变成形如['王晶 Jing Wong',26, 11, 0]的格式
    result = [[],[],[],[]]
    for j in range(4):
        result[j].append(sort_list[j][0][:9])
        result[j].extend(sort_list[j][1])
    # 写入目标字典该类型的位置
    type_director_dict[t] = result

type_director_dict

{'恐怖': [['王晶 Jing W', 26, 11, 0],
  ['杜琪峰 Johnn', 10, 16, 2],
  ['史蒂文·斯皮尔伯格', 2, 12, 11],
  ['徐克 Hark T', 11, 12, 1]],
 '悬疑': [['王晶 Jing W', 26, 11, 0],
  ['杜琪峰 Johnn', 10, 15, 2],
  ['史蒂文·斯皮尔伯格', 2, 10, 10],
  ['徐克 Hark T', 10, 11, 1]],
 '同性': [['王晶 Jing W', 26, 11, 0],
  ['杜琪峰 Johnn', 10, 16, 2],
  ['徐克 Hark T', 13, 12, 1],
  ['邱礼涛 Herma', 23, 3, 0]],
 '犯罪': [['王晶 Jing W', 25, 10, 0],
  ['徐克 Hark T', 13, 10, 1],
  ['史蒂文·斯皮尔伯格', 2, 11, 10],
  ['Woody All', 0, 11, 8]],
 '儿童': [['王晶 Jing W', 26, 11, 0],
  ['杜琪峰 Johnn', 10, 16, 2],
  ['徐克 Hark T', 13, 12, 1],
  ['邱礼涛 Herma', 23, 3, 0]],
 '喜剧': [['史蒂文·斯皮尔伯格', 1, 12, 10],
  ['邱礼涛 Herma', 19, 2, 0],
  ['徐克 Hark T', 10, 8, 1],
  ['Alfred Hi', 1, 12, 5]],
 '惊栗': [['王晶 Jing W', 26, 11, 0],
  ['杜琪峰 Johnn', 10, 16, 2],
  ['徐克 Hark T', 13, 12, 1],
  ['邱礼涛 Herma', 23, 3, 0]],
 '歌舞': [['王晶 Jing W', 26, 11, 0],
  ['杜琪峰 Johnn', 9, 16, 2],
  ['邱礼涛 Herma', 23, 3, 0],
  ['史蒂文·斯皮尔伯格', 2, 12, 11]],
 '科幻': [['王晶 Jing W', 25, 11, 0],
  ['杜琪峰 Johnn', 10, 16,

In [47]:
with open('../DataSet/douban_boutique/type_director_dict.pkl','wb') as f:
    pickle.dump(type_director_dict,f)

### 地区生成

In [3]:
zone_list = df.iloc[:,7].values

In [5]:
zone_list = zone_list.tolist()

In [7]:
zdict = {}

for z in zone_list:
    if re.search(',',z) is not None:
        continue
    if z not in zdict:
        zdict[z] = 1
    else:
        zdict[z] += 1

zdict

{'法国': 314,
 '日本': 1021,
 '美国': 3510,
 '南斯拉夫': 4,
 '香港': 1071,
 '巴西': 10,
 '德国': 91,
 '印度': 71,
 '中国大陆': 1073,
 '英国': 417,
 '韩国': 659,
 '澳大利亚': 55,
 '台湾': 157,
 '泰国': 106,
 '中国': 59,
 '俄罗斯': 42,
 '南非': 2,
 '新加坡': 12,
 '荷兰': 11,
 '加拿大': 64,
 '新西兰': 9,
 '丹麦': 10,
 '保加利亚': 1,
 '印度尼西亚': 3,
 '瑞典': 24,
 '意大利': 62,
 '西班牙': 77,
 '波兰': 23,
 '捷克斯洛伐克 Czechoslovakia': 1,
 '墨西哥': 11,
 '比利时': 4,
 '匈牙利': 7,
 '希腊': 2,
 '奥地利': 7,
 'USA': 4,
 '挪威': 12,
 '印度 India': 3,
 '伊朗': 8,
 '原西德': 3,
 '土耳其': 8,
 '苏联': 17,
 '捷克': 6,
 '瑞士': 4,
 '西德': 12,
 '前苏联': 1,
 '阿根廷': 7,
 '古巴': 1,
 '爱尔兰': 9,
 '芬兰': 6,
 '罗马尼亚': 2,
 '捷克斯洛伐克': 6,
 '爱沙尼亚': 1,
 '马来西亚': 2,
 '以色列': 5,
 '越南': 2,
 '乌克兰': 1,
 '波兰 Poland': 1,
 '加拿大 Canada': 2,
 'UK': 2,
 'South Korea': 1,
 '葡萄牙': 1,
 '新西兰 New Zealand': 1,
 '新加坡 Singapore': 1,
 '泰國': 1,
 '荷兰 Netherlands': 1,
 'Canada': 1,
 '美国/南非': 1}

In [12]:
zdict_list = zdict.items()
zdict_list

dict_items([('法国', 314), ('日本', 1021), ('美国', 3510), ('南斯拉夫', 4), ('香港', 1071), ('巴西', 10), ('德国', 91), ('印度', 71), ('中国大陆', 1073), ('英国', 417), ('韩国', 659), ('澳大利亚', 55), ('台湾', 157), ('泰国', 106), ('中国', 59), ('俄罗斯', 42), ('南非', 2), ('新加坡', 12), ('荷兰', 11), ('加拿大', 64), ('新西兰', 9), ('丹麦', 10), ('保加利亚', 1), ('印度尼西亚', 3), ('瑞典', 24), ('意大利', 62), ('西班牙', 77), ('波兰', 23), ('捷克斯洛伐克 Czechoslovakia', 1), ('墨西哥', 11), ('比利时', 4), ('匈牙利', 7), ('希腊', 2), ('奥地利', 7), ('USA', 4), ('挪威', 12), ('印度 India', 3), ('伊朗', 8), ('原西德', 3), ('土耳其', 8), ('苏联', 17), ('捷克', 6), ('瑞士', 4), ('西德', 12), ('前苏联', 1), ('阿根廷', 7), ('古巴', 1), ('爱尔兰', 9), ('芬兰', 6), ('罗马尼亚', 2), ('捷克斯洛伐克', 6), ('爱沙尼亚', 1), ('马来西亚', 2), ('以色列', 5), ('越南', 2), ('乌克兰', 1), ('波兰 Poland', 1), ('加拿大 Canada', 2), ('UK', 2), ('South Korea', 1), ('葡萄牙', 1), ('新西兰 New Zealand', 1), ('新加坡 Singapore', 1), ('泰國', 1), ('荷兰 Netherlands', 1), ('Canada', 1), ('美国/南非', 1)])

In [17]:
ok = sorted(zdict_list, key = lambda kv:kv[1], reverse=True)[:32]

ok

[('美国', 3510),
 ('中国大陆', 1073),
 ('香港', 1071),
 ('日本', 1021),
 ('韩国', 659),
 ('英国', 417),
 ('法国', 314),
 ('台湾', 157),
 ('泰国', 106),
 ('德国', 91),
 ('西班牙', 77),
 ('印度', 71),
 ('加拿大', 64),
 ('意大利', 62),
 ('中国', 59),
 ('澳大利亚', 55),
 ('俄罗斯', 42),
 ('瑞典', 24),
 ('波兰', 23),
 ('苏联', 17),
 ('新加坡', 12),
 ('挪威', 12),
 ('西德', 12),
 ('荷兰', 11),
 ('墨西哥', 11),
 ('巴西', 10),
 ('丹麦', 10),
 ('新西兰', 9),
 ('爱尔兰', 9),
 ('伊朗', 8),
 ('土耳其', 8),
 ('匈牙利', 7)]

In [18]:
ok = [r[0] for r in ok]
ok

['美国',
 '中国大陆',
 '香港',
 '日本',
 '韩国',
 '英国',
 '法国',
 '台湾',
 '泰国',
 '德国',
 '西班牙',
 '印度',
 '加拿大',
 '意大利',
 '中国',
 '澳大利亚',
 '俄罗斯',
 '瑞典',
 '波兰',
 '苏联',
 '新加坡',
 '挪威',
 '西德',
 '荷兰',
 '墨西哥',
 '巴西',
 '丹麦',
 '新西兰',
 '爱尔兰',
 '伊朗',
 '土耳其',
 '匈牙利']

In [34]:
zone_enum = ok
zone_enum

['美国',
 '中国大陆',
 '香港',
 '日本',
 '韩国',
 '英国',
 '法国',
 '台湾',
 '泰国',
 '德国',
 '西班牙',
 '印度',
 '加拿大',
 '意大利',
 '中国',
 '澳大利亚',
 '俄罗斯',
 '瑞典',
 '波兰',
 '苏联',
 '新加坡',
 '挪威',
 '西德',
 '荷兰',
 '墨西哥',
 '巴西',
 '丹麦',
 '新西兰',
 '爱尔兰',
 '伊朗',
 '土耳其',
 '匈牙利']

In [20]:
with open('../DataSet/douban_boutique/zone_set.pkl','wb') as f:
    pickle.dump(ok, f)

### zone-director

In [44]:
zone_director_dict = dict()

for t in zone_enum: # 对每种地区
    dir_dict = dict() # 清空[导演-电影数(小于7 || 7-8 || 大于8)字典]
    for i in range(df.shape[0]): # 对每一条条目
        zonestr = df.iloc[i,7] # 地区字符串
        if t!=zonestr: # 如果不是这个地区
            continue # 就跳过
        dirstr = df.iloc[i,3] # 这个电影的导演
        if type(dirstr) is not str: # 这里确保字段是字符串
            continue
        dirstr = dirstr.split(',')[0].strip()
        score = df.iloc[i,2] # 这个电影的得分
        sidx = 0 if score<7 else (1 if score<=8 else 2) # 根据得分知道要在哪个下标位置+1
        # 写入[导演-电影数字典]
        if dirstr in dir_dict:
            dir_dict[dirstr][sidx] += 1
        else:
            dir_dict[dirstr] = [0,0,0]
            dir_dict[dirstr][sidx] += 1
    # 对这个[导演-电影数]字典按电影总数从大到小排序，并只取top4
    sort_list = sorted(dir_dict.items(), key = lambda kv:(sum(kv[1])), reverse=True)[:4]
    # 将四个形如('王晶 Jing Wong', [26, 11, 0])变成形如['王晶 Jing Wong',26, 11, 0]的格式
    result = [[],[],[],[]]
    for j in range(4):
        result[j].append(sort_list[j][0][:9])
        result[j].extend(sort_list[j][1])
    # 写入目标字典该类型的位置
    zone_director_dict[t] = result

zone_director_dict

{'美国': [['史蒂文·斯皮尔伯格', 2, 9, 9],
  ['Alfred Hi', 1, 11, 5],
  ['Woody All', 0, 10, 6],
  ['伍迪·艾伦 Woo', 0, 12, 3]],
 '中国大陆': [['冯小刚 Xiaog', 4, 7, 0],
  ['张艺谋 Yimou', 3, 3, 2],
  ['海涛', 1, 5, 1],
  ['张一白 Yibai', 6, 1, 0]],
 '香港': [['王晶 Jing W', 25, 11, 0],
  ['杜琪峰 Johnn', 8, 15, 3],
  ['邱礼涛 Herma', 18, 3, 0],
  ['王晶', 17, 2, 0]],
 '日本': [['三池崇史 Taka', 14, 5, 0],
  ['园子温 Shion', 11, 5, 1],
  ['中村义洋 Yosh', 2, 9, 1],
  ['是枝裕和 Hiro', 0, 2, 9]],
 '韩国': [['洪尚秀 Sang-', 3, 11, 0],
  ['金基德 Ki-du', 1, 9, 1],
  ['柳昇完 Seung', 1, 5, 0],
  ['郑勇基 Yong-', 4, 1, 0]],
 '英国': [['爱德华·本内特 E', 0, 7, 0],
  ['安德鲁·格里弗 A', 0, 5, 1],
  ['Guy Hamil', 3, 1, 1],
  ['Alfred Hi', 1, 4, 0]],
 '法国': [['Eric Rohm', 0, 4, 9],
  ['弗朗索瓦·欧容 F', 0, 7, 1],
  ['吕克·贝松 Luc', 1, 5, 1],
  ['Jean-Luc ', 1, 3, 2]],
 '台湾': [['侯孝贤 Hsiao', 0, 2, 6],
  ['杨德昌 Edwar', 0, 2, 5],
  ['魏德圣 Te-Sh', 1, 1, 3],
  ['程伟豪 Wei-h', 2, 0, 1]],
 '泰国': [['普安农  Poj ', 5, 0, 0],
  ['班庄·比辛达拿刚 ', 0, 4, 0],
  ['由拉特·史帕克 Y', 3, 0, 0],
  ['Monthon A', 3, 0, 0]],
 '

In [45]:
with open('../DataSet/douban_boutique/zone_director_dict.pkl','wb') as f:
    pickle.dump(zone_director_dict,f)